# Regresión Ridge: Teoría y Aplicación

Este notebook aborda con profundidad la regresión Ridge, incluyendo su motivación, formulación matemática, comparación con OLS, análisis estadístico del modelo y aplicaciones modernas.

## 1. ¿Por qué regularizar?

- En regresión clásica (OLS), si los predictores están altamente correlacionados (colinealidad), la matriz $X^TX$ se vuelve casi singular.
- Esto provoca que pequeños cambios en los datos generen grandes variaciones en los coeficientes.
- Ridge penaliza los coeficientes grandes para reducir la **varianza** del estimador.

### Problemas comunes en OLS:
- Alta varianza en coeficientes
- Sobreajuste (overfitting)
- Inestabilidad numérica


## 2. Fundamento teórico
La regresión Ridge minimiza:
$$\text{min} \left\{ \| y - X\beta \|^2 + \lambda \|\beta\|_2^2 \right\}$$

Donde:
- $\|\beta\|_2^2 = \sum_j \beta_j^2$ es la norma L2
- $\lambda$ controla la intensidad de la penalización

### Solución analítica:
$$\hat{\beta}_{\text{ridge}} = (X^TX + \lambda I)^{-1} X^Ty$$

Cuando $\lambda=0$ se recupera OLS; cuando $\lambda\to\infty$ los coeficientes se acercan a cero.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.stats import shapiro

In [ ]:
# Cargar y escalar datos
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Ajustar Ridge y OLS
ridge = Ridge(alpha=10.0).fit(X_train_scaled, y_train)
ols = LinearRegression().fit(X_train_scaled, y_train)

# Predicción
y_pred_ridge = ridge.predict(X_test_scaled)
y_pred_ols = ols.predict(X_test_scaled)

# Errores
print("Ridge:", mean_squared_error(y_test, y_pred_ridge), r2_score(y_test, y_pred_ridge))
print("OLS:", mean_squared_error(y_test, y_pred_ols), r2_score(y_test, y_pred_ols))

In [ ]:
# Análisis de residuos de Ridge
residuals = y_test - y_pred_ridge

fig, ax = plt.subplots(1, 2, figsize=(12, 4))
sns.histplot(residuals, kde=True, ax=ax[0])
sm.qqplot(residuals, line='s', ax=ax[1])
ax[0].set_title("Histograma de residuos")
ax[1].set_title("Q-Q plot")
plt.tight_layout()
plt.show()

# Test de normalidad
stat, p = shapiro(residuals)
print("Shapiro-Wilk p-valor:", p)

## 3. Exploración de coeficientes vs \( \lambda \)
Vamos a observar cómo cambian los coeficientes al aumentar la regularización.

In [ ]:
alphas = np.logspace(-4, 4, 100)
coefs = []
for a in alphas:
    ridge = Ridge(alpha=a).fit(X_train_scaled, y_train)
    coefs.append(ridge.coef_)

plt.figure(figsize=(10, 6))
plt.plot(alphas, coefs)
plt.xscale("log")
plt.xlabel("lambda")
plt.ylabel("Coeficientes")
plt.title("Trayectoria de coeficientes en Ridge")
plt.grid(True)
plt.show()

## 4. Conclusión y aplicaciones
- Ridge estabiliza los coeficientes y reduce sobreajuste
- Ideal cuando hay colinealidad entre predictores
- Se usa hoy en día en genética, finanzas, modelos predictivos complejos
- Base para técnicas más avanzadas como Kernel Ridge y regularización en redes neuronales